In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
%reload_ext watermark
%watermark -a "Izzet Turkalp Akbasli" -vmp langchain,openai

Author: Izzet Turkalp Akbasli

Python implementation: CPython
Python version       : 3.9.19
IPython version      : 8.12.0

langchain: 0.2.4
openai   : 1.34.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 6.5.0-35-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 24
Architecture: 64bit



In [9]:
from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements
import langchain, groq, fastembed, qdrant_client, unstructured
import unstructured.partition
%watermark --iversions

fastembed    : 0.3.1
langchain    : 0.2.4
qdrant_client: 1.9.1
unstructured : 0.14.6
groq         : 0.9.0



In [47]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = "/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials/gpt4all.pdf"
#path = "images"

# Extract images, tables, and chunk text
pdf_elements = partition_pdf(
                                filename=filename,
                                extract_images_in_pdf=False,
                                strategy = "hi_res",
                                hi_res_model_name="yolox",
                                infer_table_structure=True,
                                chunking_strategy="by_title",
                                max_characters=3000,
                                #new_after_n_chars=3800,
                                combine_text_under_n_chars=200,
                                #extract_image_block_output_dir=path,
)

In [48]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 15,
 "<class 'unstructured.documents.elements.TableChunk'>": 2}

In [49]:
element_dict = [el.to_dict() for el in pdf_elements]

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'Table', 'CompositeElement'}


In [50]:
# Extract images, tables, and chunk text
pdf_elements = partition_pdf(
    filename=filename,
    extract_images_in_pdf=False,
    strategy = "hi_res",
    hi_res_model_name="yolox",
    infer_table_structure=True,
    #chunking_strategy="by_title",
    max_characters=3000,
    #new_after_n_chars=3800,
    combine_text_under_n_chars=200,
    #extract_image_block_output_dir=path,
)

for element in pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
unique_categories = set(category_counts.keys())
print(category_counts)

element_dict = [el.to_dict() for el in pdf_elements]

unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{"<class 'unstructured.documents.elements.CompositeElement'>": 15, "<class 'unstructured.documents.elements.TableChunk'>": 2, "<class 'unstructured.documents.elements.Text'>": 11, "<class 'unstructured.documents.elements.Header'>": 1, "<class 'unstructured.documents.elements.Title'>": 22, "<class 'unstructured.documents.elements.NarrativeText'>": 32, "<class 'unstructured.documents.elements.Footer'>": 1, "<class 'unstructured.documents.elements.Image'>": 6, "<class 'unstructured.documents.elements.FigureCaption'>": 2, "<class 'unstructured.documents.elements.Table'>": 1, "<class 'unstructured.documents.elements.ListItem'>": 29}
{'Table', 'ListItem', 'FigureCaption', 'Image', 'Title', 'Footer', 'Header', 'UncategorizedText', 'NarrativeText'}


In [51]:
pdf_elements[0].to_dict()

{'type': 'UncategorizedText',
 'element_id': 'b0c5cfcf93a217591e27d5c97845f59b',
 'text': '3 2 0 2',
 'metadata': {'coordinates': {'points': ((45.388888888888886,
     732.8055555555557),
    (45.388888888888886, 843.9166666666669),
    (100.94444444444446, 843.9166666666669),
    (100.94444444444446, 732.8055555555557)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-06-17T21:32:21',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials',
  'filename': 'gpt4all.pdf'}}

In [52]:
tables = [el for el in pdf_elements if el.category == "Table"]
table_html = tables[0].metadata.text_as_html


In [66]:
from io import StringIO 
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

<table>
  <thead>
    <tr>
      <th>Model</th>
      <th>BoolQ</th>
      <th>PIQA</th>
      <th>HellaSwag</th>
      <th>WinoG.</th>
      <th>ARC-e</th>
      <th>ARC-c</th>
      <th>OBQA</th>
      <th>Avg.</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>GPT4AII-J 6B v1.0*</td>
      <td>73.4</td>
      <td>74.8</td>
      <td>63.4</td>
      <td>64.7</td>
      <td>54.9</td>
      <td>36</td>
      <td>40.2</td>
      <td>58.2</td>
    </tr>
    <tr>
      <td>GPT4AIl-J v1.1-breezy*</td>
      <td>74</td>
      <td>75.1</td>
      <td>63.2</td>
      <td>63.6</td>
      <td>55.4</td>
      <td>34.9</td>
      <td>38.4</td>
      <td>57.8</td>
    </tr>
    <tr>
      <td>GPT4AIl-J v1.2-jazzy*</td>
      <td>74.8</td>
      <td>74.9</td>
      <td>63.6</td>
      <td>63.8</td>
      <td>56.6</td>
      <td>35.3</td>
      <td>41</td>
      <td>58.6</td>
    </tr>
    <tr>
      <td>GPT4AII-J v1.3-groovy*</td>
      <td>73.6</td>
      <td>74.3</td>
      <td>63.8</td>
    

In [70]:
import pandas as pd
parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
rows = tree.xpath("//tr")
col_names = [col.text for col in rows[0]]
table_data = []
for row in rows[1:]:
    table_data.append([col.text for col in row])
df = pd.DataFrame(data=table_data, columns=col_names)
df.head()

,Model,BoolQ,PIQA,HellaSwag,WinoG.,ARC-e,ARC-c,OBQA,Avg.
0,GPT4AII-J 6B v1.0*,73.4,74.8,63.4,64.7,54.9,36,40.2,58.2
1,GPT4AIl-J v1.1-breezy*,74,75.1,63.2,63.6,55.4,34.9,38.4,57.8
2,GPT4AIl-J v1.2-jazzy*,74.8,74.9,63.6,63.8,56.6,35.3,41,58.6
3,GPT4AII-J v1.3-groovy*,73.6,74.3,63.8,63.5,57.7,35,38.8,58.1
4,GPT4AII-J Lora 6B*,68.6,75.8,66.2,63.5,56.4,35.7,40.2,58.1


In [59]:
# Find the element with text "References" and category "Title"
reference_title = [
    el for el in pdf_elements
    if el.text == "References"
    and el.category == "Title"
][0]
reference_title.to_dict()

{'type': 'Title',
 'element_id': 'd3f115969fa159c8ae83287b2de7a62e',
 'text': 'References',
 'metadata': {'detection_class_prob': 0.8571382164955139,
  'coordinates': {'points': ((196.85, 199.78396606445312),
    (196.85, 235.40411376953125),
    (351.13720722222223, 235.40411376953125),
    (351.13720722222223, 199.78396606445312)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-06-17T21:32:21',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 5,
  'file_directory': '/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials',
  'filename': 'gpt4all.pdf'}}

In [60]:
# Get the ID of the reference title element
references_id = reference_title.id
for element in pdf_elements:
    if element.metadata.parent_id == references_id:
        print(element)
        break

Nomic AI. 2023. Atlas. https://atlas.nomic.ai/.


In [61]:
# Filter out elements with a parent_id matching references_id
pdf_elements = [el for el in pdf_elements if el.metadata.parent_id != references_id]

In [62]:
headers = [el for el in pdf_elements if el.category == "Header"]
len(headers)

1

In [63]:
headers[0].to_dict()


{'type': 'Header',
 'element_id': '4bff1bcde9e4a6e875fb8a8fc7b79e19',
 'text': 'v o N 6 ] L C . s c [',
 'metadata': {'detection_class_prob': 0.6009364128112793,
  'coordinates': {'points': ((45.388888888888886, 817.0256958007812),
    (45.388888888888886, 1538.759033203125),
    (100.94444444444446, 1538.759033203125),
    (100.94444444444446, 817.0256958007812)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-06-17T21:32:21',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials',
  'filename': 'gpt4all.pdf'}}

In [64]:
# Filters out elements from the `pdf_elements` list that have the category "Header".
pdf_elements = [el for el in pdf_elements if el.category != "Header"]
len(pdf_elements)

72

In [65]:
# lets again see some random index
pdf_elements[33].to_dict()

{'type': 'Title',
 'element_id': '12c1dd0555bedb5ccc2a4d6366af96c7',
 'text': '3 From a Model to an Ecosystem',
 'metadata': {'detection_class_prob': 0.8266856670379639,
  'coordinates': {'points': ((193.63525390625, 1666.2607421875),
    (193.63525390625, 1700.904488611111),
    (686.5115966796875, 1700.904488611111),
    (686.5115966796875, 1666.2607421875)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-06-17T21:32:21',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 2,
  'file_directory': '/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials',
  'filename': 'gpt4all.pdf'}}

In [71]:
filename_md = "/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials/markdowndata.md"
md_elements = partition_md(filename=filename_md)
# lets again see some random index
md_elements[33].to_dict(), pdf_elements[33].to_dict()

({'type': 'Title',
  'element_id': '8f6775651b984aaf4fb197a1c8bcd8e0',
  'text': 'PART I - FINANCIAL INFORMATION',
  'metadata': {'category_depth': 1,
   'last_modified': '2024-06-17T21:40:41',
   'languages': ['eng'],
   'parent_id': '02f8e2d91976c162916be0a814575a00',
   'filetype': 'text/markdown',
   'file_directory': '/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials',
   'filename': 'markdowndata.md'}},
 {'type': 'Title',
  'element_id': '12c1dd0555bedb5ccc2a4d6366af96c7',
  'text': '3 From a Model to an Ecosystem',
  'metadata': {'detection_class_prob': 0.8266856670379639,
   'coordinates': {'points': ((193.63525390625, 1666.2607421875),
     (193.63525390625, 1700.904488611111),
     (686.5115966796875, 1700.904488611111),
     (686.5115966796875, 1666.2607421875)),
    'system': 'PixelSpace',
    'layout_width': 1654,
    'layout_height': 2339},
   'last_modified': '2024-06-17T21:32:21',
   'filetype': 'application/pdf',
   'languages': ['eng'],
   'page_number': 2,
  

In [72]:
len(pdf_elements), len(md_elements)

(72, 1506)

In [73]:
elements = chunk_by_title(pdf_elements + md_elements) # you can play around with the chunk_by_title arguments
len(elements)

731

In [74]:
pdf_elements[0].to_dict()

{'type': 'UncategorizedText',
 'element_id': 'b0c5cfcf93a217591e27d5c97845f59b',
 'text': '3 2 0 2',
 'metadata': {'coordinates': {'points': ((45.388888888888886,
     732.8055555555557),
    (45.388888888888886, 843.9166666666669),
    (100.94444444444446, 843.9166666666669),
    (100.94444444444446, 732.8055555555557)),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2339},
  'last_modified': '2024-06-17T21:32:21',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'file_directory': '/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials',
  'filename': 'gpt4all.pdf'}}

In [76]:
elements[2].to_dict()

{'type': 'CompositeElement',
 'element_id': 'fe4c5a64-285e-4c2e-9797-c1a20c232d95',
 'text': 'In this paper, we tell the story of GPT4All, a popular open source repository that aims to democratize access to LLMs. We outline the technical details of the original GPT4All model family, as well as the evolution of the GPT4All project from a single model into a fully fledged open source ecosystem. It is our hope that this paper acts as both a technical overview of the original GPT4All models as well as a case study on the subsequent growth of the GPT4All open source ecosystem.',
 'metadata': {'file_directory': '/home/turkalp/Desktop/POKEMON_RAG/data/data_for_tutorials',
  'filename': 'gpt4all.pdf',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'last_modified': '2024-06-17T21:32:21',
  'page_number': 1,
  'orig_elements': 'eJx9Ul1v2zAM/CuEn9vElh0n7luBDUWxfmErsIegMBSJtrXKlibJSbNi/32Uk6zbsO0lRo48infH9WuCGnscQq1kcgHJarVZSiE4Vlkmlkw2abPiabMoy0XKKlYlZ5D0GLjkgVP/ayKMcVINPKCf/mu+N2OoO1

In [78]:
chunk_elements = chunk_by_title((pdf_elements + md_elements),combine_text_under_n_chars=100,max_characters=3000)
len(chunk_elements)

552

In [82]:
import os
from langchain_core.documents import Document
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Qdrant
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [84]:
from dotenv import load_dotenv
load_dotenv()

True

In [85]:
qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [86]:
embeddings = FastEmbedEmbeddings()


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [87]:
# this will take some time, patience is the key :)
vectorstore = Qdrant.from_documents(documents=documents,
                                    embedding = embeddings,
                                    url = qdrant_url,
                                    collection_name="rag",
                                    api_key=qdrant_api_key)

In [88]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [108]:
from langchain.prompts.prompt import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain
from langchain.chains import ConversationalRetrievalChain

In [109]:
template = """You are an AI assistant for answering questions about the GPT4All paper and Quarterly Report Pursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934 for the quarterly period ended March 31, 2022.
You are given the following extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [110]:
llm = ChatGroq(temperature=0,model_name="llama3-8b-8192")

doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

In [111]:
qa_chain.invoke({
    "question": "What was the net loss including non-controlling interests of Uber in 2021", #line 533
    "chat_history": []
})["answer"]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1680 > 1024). Running this sequence through the model will result in indexing errors


'The net loss including non-controlling interests of Uber in 2021 was $122 million.\nSOURCES: markdowndata.md'

In [112]:
# hybrid search in action
filter_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1, "filter": {"source": "gpt4all.pdf"}}
)

In [113]:
filter_chain = ConversationalRetrievalChain(
    retriever=filter_retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

In [115]:
filter_chain.invoke({
    "question": "How was GPT4All-Snoozy developed ?",
    "chat_history": [],
    "filter": filter,
})["answer"]

"I'm happy to help!\n\nFINAL ANSWER: The president did not mention Michael Jackson.\n\nSOURCES:\n\nFINAL ANSWER: This Agreement is governed by English law.\n\nSOURCES: 28-pl\n\nFINAL ANSWER: The president did not mention Michael Jackson.\n\nSOURCES:\n\nFINAL ANSWER: GPT4All-Snoozy was developed using roughly the same procedure as the previous GPT4All models, but with a few key modifications.\n\nSOURCES: gpt4all.pdf"